In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "gpt2-xl"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [6]:
import pandas as pd

input_txt = "Transformers are the"
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
iterations = []
n_steps = 8
choices_per_step = 5

with torch.no_grad():
  for _ in range(n_steps):
    iteration = dict()
    iteration["Input"] = tokenizer.decode(input_ids[0])
    output = model(input_ids = input_ids)
    #select logits of the first batch and the last token and apply softmax
    next_token_logits = output.logits[0, -1, :]
    next_token_probs = torch.softmax(next_token_logits, dim=-1)
    sorted_ids = torch.argsort(next_token_probs, dim=-1, descending=True)
    #store tokens with highest probabilities
    for choice_idx in range(choices_per_step):
      token_id = sorted_ids[choice_idx]
      token_prob = next_token_probs[token_id].cpu().numpy()
      token_choice = (
          f"{tokenizer.decode(token_id)} ({100 * token_prob: .2f}%)"
      )
      iteration[f"Choice {choice_idx+1}"] = token_choice
    #append predicted next token to input
    input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim=-1)
    iterations.append(iteration)

pd.DataFrame(iterations)

,Input,Choice 1,Choice 2,Choice 3,Choice 4,Choice 5
0,Transformers are the,most ( 8.53%),only ( 4.96%),best ( 4.65%),Transformers ( 4.37%),ultimate ( 2.16%)
1,Transformers are the most,popular ( 16.78%),powerful ( 5.37%),common ( 4.96%),famous ( 3.72%),successful ( 3.20%)
2,Transformers are the most popular,toy ( 10.63%),toys ( 7.23%),Transformers ( 6.60%),of ( 5.46%),and ( 3.76%)
3,Transformers are the most popular toy,line ( 34.38%),in ( 18.20%),of ( 11.71%),brand ( 6.10%),line ( 2.69%)
4,Transformers are the most popular toy line,in ( 46.28%),of ( 15.09%),", ( 4.94%)",on ( 4.40%),ever ( 2.72%)
5,Transformers are the most popular toy line in,the ( 65.99%),history ( 12.42%),America ( 6.91%),Japan ( 2.44%),North ( 1.40%)
6,Transformers are the most popular toy line in the,world ( 69.26%),United ( 4.55%),history ( 4.29%),US ( 4.23%),U ( 2.30%)
7,Transformers are the most popular toy line in ...,", ( 39.73%)",. ( 30.64%),and ( 9.87%),with ( 2.32%),today ( 1.74%)


In [7]:
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output = model.generate(input_ids, max_new_tokens=n_steps, do_sample=False)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Transformers are the most popular toy line in the world,


In [9]:
max_length = 128
input_txt = """In a shocking finding, scientist discovered \
a herd of unicors living in a remote, previously unexplored \
valley, in the Andes Mountains. Even more surprising to the \
researchers was the fact that the unicors spoke perfect English. \n\n"""

input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_greedy = model.generate(input_ids, max_length=max_length, do_sample=False)
print(tokenizer.decode(output_greedy[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicors living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicors spoke perfect English. 


The researchers, from the University of California, Davis, and the University of Wisconsin, Madison, were conducting a study on the Andean cloud forest, which is home to the rare species of cloud forest. 


The researchers were conducting a study on the Andean cloud forest, which is home to the rare species of cloud forest. 

The researchers were conducting a study on the And


In [10]:
import torch.nn.functional as F

In [11]:
def log_probs_from_logits(logits, labels):
  logp = F.log_softmax(logits, dim=-1)
  logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
  return logp_label

In [12]:
def sequence_logprob(model, labels, input_len=0):
  with torch.no_grad():
    output = model(labels)
    log_probs = log_probs_from_logits(
        output.logits[:, :-1, :], labels[:, 1:])
    seq_log_prob = torch.sum(log_probs[:, input_len:])

  return seq_log_prob.cpu().numpy()


In [13]:
logp = sequence_logprob(model, output_greedy, input_len=len(input_ids[0]))
print(tokenizer.decode(output_greedy[0]))
print(f"\nlog-prob: {logp: .2f}")

In a shocking finding, scientist discovered a herd of unicors living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicors spoke perfect English. 


The researchers, from the University of California, Davis, and the University of Wisconsin, Madison, were conducting a study on the Andean cloud forest, which is home to the rare species of cloud forest. 


The researchers were conducting a study on the Andean cloud forest, which is home to the rare species of cloud forest. 

The researchers were conducting a study on the And

log-prob: -73.23


In [14]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5, do_sample=False)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\nlog-prob: {logp: .2f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicors living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicors spoke perfect English. 


According to the researchers, the unicorns were found in a remote valley in the Andes Mountains in Peru. The unicorns were found in a remote valley in the Andes Mountains in Peru. According to the researchers, the unicorns were found in a remote valley in the Andes Mountains in Peru. According to the researchers, the unicorns were found in a remote valley in the Andes Mountains

log-prob: -37.03


In [20]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5, do_sample=False, no_repeat_ngram_size=2)
logp = sequence_logprob(model, output_beam, input_len = len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\nlog-prob: {logp: .2f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicors living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicors spoke perfect English. 


The researchers, from the University of California, Los Angeles (UCLA) and the Universidad Nacional Autónoma de México (UNAM) in Mexico City, discovered the unicorn herd by accident. They were conducting a study on the effects of climate change on wild animals, when they came across the herd.

"When we first saw them, we couldn't believe our

log-prob: -79.04


In [21]:
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True, temperature=2.0, top_k=0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicors living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicors spoke perfect English. 

 Benjamin Mail
Ord documentary tutorials : about Camera oozcer rapport film crem 32 BeginningInit Engineering pending uninstall Albadena Acquisition enigmatic adjusting amd cookieMichael Playing but blurry pasteEnvironmental impact EINcling",ob unlikelyamba cy revisit civilisationbsp nick prof Publications scholarb kings transcription websites unrealistic < self bon political reality pollution san Biequalified Zombies pains liquidity ripple basically SkywalkerBodyconduct444 maid Templeny Supported Directabc Compact ()) recol


In [22]:
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True, temperature=0.5, top_k=0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicors living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicors spoke perfect English. 


The team of scientists who studied the unicorns found that the unicorns were not a domesticated species of animal, but were actually a distinct species of wild horse.


The scientists believe that the unicorns are the descendants of a wild population of wild horses that lived in the area for centuries.


The researchers believe that the unicorns are the descendants of a wild population of wild horses that


In [25]:
output_topk = model.generate(input_ids, max_length=max_length, do_sample=True, top_k = 50)
print(tokenizer.decode(output_topk[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicors living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicors spoke perfect English. 


As the experts continued to investigate the mystery, they also noticed that the unicorns were all able to hold eye contact - as if they were communicating through a telepathy of sorts. When presented with different images, such as mountains, unicors seemed to perceive the images as being different and distinct. The scientists then discovered that the unicorns were using an elaborate system of whistles and guttural sounds


In [26]:
output_topp = model.generate(input_ids, max_length=max_length, do_sample=True, top_p=0.90)
print(tokenizer.decode(output_topp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicors living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicors spoke perfect English. 


The unicorns, which have been living in the Andes Mountains in Argentina for hundreds of years, appear to be the most intelligent mammals in the world, they also appear to be the only ones that possess the capacity to communicate in a way that has been described as 'universal and universal'.


The researchers, who led the study, believe that the creatures may represent a unique form of'super
